In [1]:
day = 8
from IPython.display import IFrame
url = "https://adventofcode.com/2022/day/%d"%(day)
IFrame(src=url, width=1000, height=600)

In [2]:
import urllib.request
import sys
from collections import defaultdict

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2022):
    with open('sessionID') as f:
        sessionId = f.readlines()[0]
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data

def visible_idx(row):
    out = []
    idx =-1
    val_max = -1
    done = False
    while not done:
        mask = row > val_max
        new_idx = np.where(mask==True)[0]
        if len(new_idx) == 0:
            done = True
        else:
            idx = new_idx[0]
            out.append(idx)
            val_max = row[idx]

    return out 

def mask_hidden(tree_map, visible):
    for ii, row in enumerate(tree_map):
        visible[ii, visible_idx(row)] = 1
    return visible

def map_and_mask (data):
    temp = []
    for line in data:
        temp.append([int(val) for val in line])
    data = np.array(temp)
    visible = np.zeros_like(data)
    return data, visible


def score_map(tree_map, score_map):
    for ii, row in enumerate(tree_map):
        score_map[ii, :] = score_map[ii, :]*view_score(row)
    return score_map

def view_score(row):
    out = np.zeros_like(row)
    temp = row.copy()
    temp[-1] = 11
    
    for ii, val in enumerate(temp):
        score = np.where(temp[ii:]>=val)[0]        
        if len(score) > 1:
             out[ii] = score[1]
    return out 



In [3]:
from timeit import default_timer as timer
import numpy as np

start = timer()
data = get_data(day)
data, visible = map_and_mask(data)

visible = mask_hidden(data, visible)
visible = mask_hidden(np.fliplr(data), np.fliplr(visible))
visible = np.fliplr(visible)
visible = mask_hidden(data.T, visible.T)
visible = visible.T
visible = mask_hidden(np.fliplr(data.T), np.fliplr(visible.T))
part1 = np.sum(visible)

print("Completed in %fms" % ((timer() - start) * 1000))
print(part1)


start = timer()
data = get_data(day)


data, score = map_and_mask(data)
score = score + 1

score_left = score.copy()
score_left = score_map(data, np.ones_like(score_left))
score_right = score.copy()
score_right = np.fliplr(score_map(np.fliplr(data), np.fliplr(score_right)))
score_top = score.copy()
score_top = (score_map(data.T, score_top.T)).T
score_bottom = score.copy()
score_bottom = score_map(np.fliplr(data.T), np.fliplr(score_bottom.T))
score_bottom = np.fliplr(score_bottom).T
part2 = np.max(score_left * score_right * score_top * score_bottom)

print(part1, part2)

Completed in 193.455754ms
1662
1662 537600
